In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Neural Network

In [ ]:
class NeuralNetwork:
    W = []
    M = []
    b = []
    Yhat = []
    costs = []
    actf = 0
    
    def __sigmoid(self, Z):
        return 1 / (1 + np.exp(-Z))
    
    def __tanh(self, Z):
        return np.tanh(Z)
    
    def __relu(self,Z):
        return Z * (Z > 0)
    
    # Calls specified activation function
    def __sig(self,Z,ty):
        if ty == 0:
            return self.__sigmoid(Z)
        if ty == 1:
            return self.__tanh(Z)
        else:
            return self.__relu(Z)
        
        # Calls specified activation function derivative
    def __sig_dv(self,Z,ty):
        if ty == 0:
            return Z*(1-Z)
        if ty == 1:
            return (1-Z*Z)
        else:
            return np.where(Z > 0, 1, 0)
        
    def __softmax(self,A):
        expA = np.exp(A)
        return expA / expA.sum(axis=1,keepdims=True)
    
    # Cost for softmax
    def __cost(self,T,Y):
        tot = (-T * np.log(Y))
        return tot.sum()
    
    def predict(self,Py):
        return np.argmax(Py, axis=1)
    
    def classification_rate(self,T,Y):
        return np.mean(T == Y)
    
    def __forward(self,M,W,b,actf):
        for i in range(1,len(W)+1):
            if (i != len(W)):
                M[i] = self.__sig((M[i-1].dot(W[i-1]) + b[i-1]), actf)
            else:
                M[i] = M[i-1].dot(W[i-1]) + b[i-1]
                Y = self.__softmax(M[i])
        return Y,M